# Talk about Framing and outlining the problem (Business problem and specifics)
# Talk about data

In [2]:
# All of your imports here (you may need to add some)
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import pyarrow as pa
import fastparquet as fp

In [3]:
def read_csv(file_path):
    """Reads a CSV file and returns its content as a Pandas DataFrame."""
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return None


read_csv('T_ONTIME_REPORTING.csv')

/var/folders/rp/qmfm4r4n4w16dpc3gqbtt_lm0000gp/T/ipykernel_60510/3866758592.py:4: DtypeWarning: Columns (77,84,85,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DIV4_WHEELS_OFF,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM
0,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N131EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N132EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N132EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N133EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024,1,1,1,1,1/1/2024 12:00:00 AM,9E,20363,9E,N133EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547266,2024,1,1,31,3,1/31/2024 12:00:00 AM,YX,20452,YX,N879RW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547267,2024,1,1,31,3,1/31/2024 12:00:00 AM,YX,20452,YX,N882RW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547268,2024,1,1,31,3,1/31/2024 12:00:00 AM,YX,20452,YX,N882RW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
547269,2024,1,1,31,3,1/31/2024 12:00:00 AM,YX,20452,YX,N882RW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
